In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

Considering the possible effects connected to LC50. We need to filter on **Mortality Group**

## Preprocessing on base_db

In [2]:
base_db = pd.read_csv("use_db_sabato14.csv")
base_db.head()

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,10108-64-2,S,48.0,T,46.900,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
1,10108-64-2,S,24.0,T,200.000,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Fundulus,heteroclitus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
2,10108-64-2,F,96.0,T,0.001,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
3,10108-64-2,S,96.0,T,15.900,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Cyprinodon,variegatus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
4,10108-64-2,F,694.0,T,0.030,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945


In [3]:
base_db = base_db[base_db.obs_duration_mean.isin([24, 48, 72, 96])]

In [4]:
to_drop = base_db[base_db.bonds_number.isnull()].index

In [5]:
base_db = base_db.drop(to_drop,axis=0)

In [6]:
base_db.index.size

45775

In [7]:
base_db.columns

Index(['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species',
       'atom_number', 'alone_atom_number', 'tripleBond', 'doubleBond',
       'bonds_number', 'ring_numer', 'Mol', 'MorganDensity', 'LogP'],
      dtype='object')

In [8]:
use_db = base_db.copy()
use_db = use_db[['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species','atom_number', 'alone_atom_number', 'tripleBond',
       'doubleBond', 'bonds_number', 'ring_numer',
       'Mol', 'MorganDensity', 'LogP']]

In [9]:
use_db.index.size

45775

# Repeated experiments

In [10]:
db_species = use_db[["species", 'class', 'tax_order', 'family', 'genus']]

In [11]:
use_db = use_db.groupby(by=["test_cas", "species", "conc1_type", "exposure_type", "obs_duration_mean",]).agg("median").reset_index()
use_db.index.size

23332

In [12]:
db_species = db_species.groupby("species").first()

In [13]:
use_db = use_db.merge(db_species, on="species")

In [14]:
use_db

,test_cas,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP,class,tax_order,family,genus
0,100-00-5,rerio,F,F,48.0,15.00,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio
1,100-00-5,rerio,F,F,96.0,15.00,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio
2,100-01-6,rerio,F,S,96.0,87.60,0.317908,2,0,1,0.488106,1.0,0.072092,1.300000,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Danio
3,100-02-7,rerio,F,AQUA,48.0,26.50,0.317908,2,0,1,0.488106,1.0,0.072622,1.300000,1.3004,Actinopterygii,Cypriniformes,Cyprinidae,Danio
4,100-44-7,rerio,F,S,96.0,4.00,0.217006,0,0,0,0.447258,1.0,0.065856,1.125000,2.4254,Actinopterygii,Cypriniformes,Cyprinidae,Danio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23327,94-75-7,boga,F,R,96.0,3.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Labeo
23328,94-75-7,neilgherriensis,F,R,24.0,7.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora
23329,94-75-7,neilgherriensis,F,R,48.0,6.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora
23330,94-75-7,neilgherriensis,F,R,96.0,5.60,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora


### Simple classification with Smile Features
Considering a threshold of 1mg/L of mortality

In [49]:
X = use_db.copy()
X = X[[
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species',
    'class', 'tax_order', 'family', 'genus',
 'atom_number', 
    'alone_atom_number', 
    'tripleBond',
       'doubleBond',
 'bonds_number',
  'ring_numer',
  'Mol', 
  'MorganDensity', 'LogP']]

In [50]:
X['exposure_type'].unique()

array(['F', 'S', 'AQUA', 'R', 'E', 'SP', 'P', 'O', 'IP', 'FD'],
      dtype=object)

In [51]:
X['conc1_type'].unique()

array(['F', 'D', 'T', 'A', 'U', 'L'], dtype=object)

In [52]:
len(X['species'].unique())

475

For this  vaiables we decided to transform than by an OrdinalEncoder, in order to have all numnerical variables

In [53]:
enc = OrdinalEncoder(dtype=int)
enc.fit(X[["exposure_type", "conc1_type","species", 'class', 'tax_order', 'family', 'genus']])
X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']] = enc.transform(X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']]) + 1

In [54]:
X.head()

,exposure_type,obs_duration_mean,conc1_type,species,class,tax_order,family,genus,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,3,48.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.082505,1.300,2.2482
1,3,96.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.082505,1.300,2.2482
2,9,96.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.072092,1.300,1.1770
3,1,48.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.072622,1.300,1.3004
4,9,96.0,3,379,1,10,35,80,0.217006,0,0,0,0.447258,1.0,0.065856,1.125,2.4254


We also devide the label, in order to have a binary classification

In [55]:
y = use_db["conc1_mean"].copy().values
y = np.where(y > 1, 1, 0)

In [56]:
len(X)

23332

In [57]:
print(len(y[y==0]), len(y[y==1]))

8566 14766


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## KNN

We first implement a classification by a K Neighbors Classifier

In [83]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [84]:
accuracy_score(y_test, y_pred)

0.7332467532467533

## FROM HERE WE USE THE DISTANCE MATRIX

In [59]:
categorical = [
'ring_numer',
"exposure_type", 
"conc1_type","species",
'tripleBond',
'obs_duration_mean',
'doubleBond',
'alone_atom_number',
'class', 'tax_order', 'family', 'genus'
 ]

In [60]:
non_categorical =[
 'atom_number',
 'bonds_number',
 'Mol',
 'MorganDensity',
 'LogP']

In [61]:
len(categorical)

12

In [62]:
len(categorical) + len(non_categorical)

17

In [63]:
X_try  =  X#.head(30000)
y_try = y#[:30000]

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_try, y_try, test_size=0.33, random_state=42)
X_try = X_train.append(X_test)

In [65]:
import numpy as np
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def new_distance_matrix(X, len_X_train, cat_features = [], num_features = [], alpha = 1):
    ''' inputs: matrix X [num_samples, num_features], the list of the categorical features, the list of the numerical features, weight alpha
        output: distance matrix
    '''

    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
    dist_matr += squareform(pdist(X_num, metric = "euclidean"))

    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test


In [30]:
# len_X_train = len(X_train)
# X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = 5)

In [28]:
X_train_new.shape

(15647, 15647)

In [29]:
X_test_new.shape

(7708, 15647)

In [30]:
neigh = KNeighborsClassifier(metric = 'precomputed')
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

In [31]:
accuracy_score(y_test, y_pred)

0.8609237156201349

## Best features analysis

In [31]:
import numpy as np
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def new_distance_matrix_selective(X, len_X_train, cat_features = [], num_features = [], alpha = 1):
    ''' inputs: matrix X [num_samples, num_features], the list of the categorical features, the list of the numerical features, weight alpha
        output: distance matrix
    '''

    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    
    if (len(cat_features)!=0 and len(num_features)):
        dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
        dist_matr += squareform(pdist(X_num, metric = "euclidean"))
    elif(len(cat_features)!=0):
        dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
    else:
        dist_matr = squareform(pdist(X_num, metric = "euclidean"))

    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test


In [32]:
import itertools
np.random.seed(13)

best_acc = 0
best_comb = []
best_cat = []
best_non_cat = []

poss_features = np.array(categorical + non_categorical)
for k in range(1, 18):
    print("Starting k =",k)
    poss_comb = list(itertools.combinations(range(0,17),k))
    for c in poss_comb:
        cat = []
        non_cat = []
    for i in list(c):
        if i in range(0, 12):
            cat.append(poss_features[i])
        else:
            non_cat.append(poss_features[i])

    len_X_train = len(X_train)
    X_train_new, X_test_new = new_distance_matrix_selective(X_try, len_X_train, cat,non_cat, alpha = 1)
    
    neigh = KNeighborsClassifier(metric = 'precomputed')
    neigh.fit(X_train_new, y_train.ravel())
    y_pred = neigh.predict(X_test_new)
    
    acc = accuracy_score(y_test, y_pred)

    if acc>best_acc:
        best_acc = acc
        best_cat = cat
        best_non_cat = non_cat
        print("Best combination found! Acc: {}, features: cat: {}, non_cat:{}".format(best_acc, best_cat, best_non_cat))
    
    del X_train_new, X_test_new

Starting k = 1
Best combination found! Acc: 0.814608199273482, features: cat: [], non_cat:['LogP']
Starting k = 2
Best combination found! Acc: 0.8319927348209653, features: cat: [], non_cat:['MorganDensity', 'LogP']
Starting k = 3
Best combination found! Acc: 0.8423715620134925, features: cat: [], non_cat:['Mol', 'MorganDensity', 'LogP']
Starting k = 4
Best combination found! Acc: 0.8458744161909704, features: cat: [], non_cat:['bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 5
Best combination found! Acc: 0.8473015049299429, features: cat: [], non_cat:['atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 6
Starting k = 7
Starting k = 8
Starting k = 9
Starting k = 10
Starting k = 11
Best combination found! Acc: 0.8484691229891023, features: cat: ['doubleBond', 'alone_atom_number', 'class', 'tax_order', 'family', 'genus'], non_cat:['atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 12
Best combination found! Acc: 0.861312921639

## CV

In [66]:
from sklearn.model_selection import KFold

def cv_new_distance(X, y, cat_features = [], num_features = [], alphas = [], ks = [], leafs=[]):
    np.random.seed(13)
    best_accuracy = 0
    best_alpha = 0
    best_k = 0
    best_leaf = 0
    
    X_cat = X[cat_features]
    X_num = X[num_features]
    dist_matr_num = squareform(pdist(X_num, metric = "euclidean"))


    for alpha in alphas:
        for k in ks:
            for leaf in leafs:
                
                dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
                dist_matr += dist_matr_num
                dist_matr = pd.DataFrame(dist_matr)

                kf = KFold(n_splits=5, shuffle=True)
                accs = []
                for train_index, test_index in kf.split(dist_matr):
                    
                    X_train = dist_matr.iloc[train_index, train_index]
                    X_test = dist_matr.iloc[test_index, train_index]
                    y_train = y[train_index]
                    y_test = y[test_index]

                    neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=k, n_jobs=-2, leaf_size=leaf)
                    neigh.fit(X_train, y_train.ravel())
                    y_pred = neigh.predict(X_test)

                    accs.append(accuracy_score(y_test, y_pred))

                avg_acc = np.mean(accs)
                if (avg_acc > best_accuracy):
                    print("New best params found! alpha:{}, k:{}, leaf:{}, acc:{}".format(alpha, k, leaf, avg_acc))
                    best_alpha = alpha
                    best_k = k
                    best_accuracy = avg_acc
                    best_leaf = leaf

    return best_accuracy, best_alpha, best_k, best_leaf


In [67]:
alphas = np.logspace(-3, 0, 30)
ks = range(1, 5)
leafs = range(30, 101, 10)
best_acc, best_alpha, best_k, best_leaf = cv_new_distance(X_train, y_train, categorical, non_categorical, alphas, ks, leafs)

New best params found! alpha:0.001, k:1, leaf:30, acc:0.8934877353477779
New best params found! alpha:0.001, k:1, leaf:40, acc:0.8957908960018626
New best params found! alpha:0.001, k:1, leaf:50, acc:0.8969423433366049
New best params found! alpha:0.001, k:1, leaf:90, acc:0.8979019339331081
New best params found! alpha:0.0016102620275609393, k:1, leaf:60, acc:0.8984773609253482


KeyboardInterrupt: 

# First test (No feature selection)

In [28]:
best_alpha = 0.3359818286283781

In [29]:
best_k = 3

In [30]:
best_leaf = 50

In [31]:
len_X_train = len(X_train)
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = best_alpha)
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=best_k, leaf_size=best_leaf)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

In [32]:
accuracy_score(y_test, y_pred)

0.8779220779220779

In [33]:
from math import sqrt
from sklearn.metrics import mean_squared_error
sqrt(mean_squared_error(y_test, y_pred))

0.3493965112560829

# Second test (all 17 features)

In [30]:
# range used
# alphas = np.logspace(-1, 2, 30)
# ks = range(1, 5)
# leafs = range(30, 101, 10)
# best_acc, best_alpha, best_k, best_leaf = cv_new_distance(X_train, y_train, categorical, non_categorical, alphas, ks, leafs)

In [31]:
best_alpha = 0.0016102620275609393
best_leaf = 70
best_k = 1

In [34]:
len_X_train = len(X_train)
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = best_alpha)
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=best_k, leaf_size=best_leaf)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

In [35]:
accuracy_score(y_test, y_pred)

0.9002597402597402